In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, Flatten, BatchNormalization, Lambda

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
def create_dynamic_cnn(input_dim, num_classes, sliding_window_size=3):
    """
    Creates a CNN dynamically based on input dimensions and label classes,
    with a decreasing number of filters in successive layers.
    """
    # Input layer
    input_layer = Input(shape=(input_dim,))
    
    # Reshape input to be compatible with Conv1D using Lambda layer
    reshaped_input = Lambda(lambda x: tf.expand_dims(x, axis=-1))(input_layer)  # Shape: (batch_size, input_dim, 1)
    
    # Determine number of convolutional layers based on input size
    num_conv_layers = max(2, input_dim // 10)  # At least 2 layers
    filters = input_dim * 2  # Start with a reasonable number of filters
    x = reshaped_input

    # Add convolutional layers with decreasing filter counts
    for _ in range(num_conv_layers):
        x = Conv1D(filters=filters, kernel_size=sliding_window_size, activation='relu', padding='same', strides=1)(x)
        x = BatchNormalization()(x)
        x = Dropout(0.2)(x)
        filters = int(max(16, filters // 2))  # Gradually decrease filters, but not below 16
    
    # Flatten the output of the final Conv1D layer
    x = Flatten()(x)
    
    # Fully connected layers
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    
    # Create model
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [5]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        train_data['label'] = train_data['label'].map(label_mapping)
    train_X = train_data.drop(columns=['label']).values
    train_y = train_data['label'].values
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))
    del train_data
    gc.collect()

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))
    del test_data
    gc.collect()

    # EarlyStopping Callback (optional, to avoid overfitting)
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

    # Number of runs for averaging results
    num_runs = 5

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Train the Model with Validation Split N tines for more accurate metrics
    #print("Verbose output only for first run...")
    verbose_run = 1
    for run in range(num_runs):
        
        # Model is defined separately in each run, since the random combination layers
        # must be randomly initialized each time. Otherwise, the "random" indices stay the same
        # throughout all runs
        model = create_dynamic_cnn(train_X.shape[1], test_y.shape[1])

        print(f"Run {run + 1}/{num_runs} started...")
        history = model.fit(
            train_X, train_y, 
            epochs=1000, 
            batch_size=int(train_X.shape[0] * 0.01),
            callbacks=[early_stopping],
            verbose=verbose_run
        )
        verbose_run = 0 # Suppress detailed output for multiple runs

        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        y_pred = model.predict(test_X, verbose=0)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = test_y.argmax(axis=1)
        del model
        gc.collect()

        # Compute metrics
        balanced_acc = balanced_accuracy_score(y_true_classes, y_pred_classes)
        roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')  # `test_y` is fine here for AUC
        pr_auc = average_precision_score(test_y, y_pred, average='weighted')
        f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
        mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

        # Store metrics
        metrics_storage['test_loss'].append(test_loss)
        metrics_storage['test_accuracy'].append(test_acc)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

    # Average the metrics over all runs
    print("\nAggregated Metrics:")
    for metric, values in metrics_storage.items():
        avg_value = np.mean(values)
        print(f"{metric}: {avg_value:.4f}")

    gc.collect()
        

In [6]:
run_model("covtype_train.csv", "covtype_test.csv", is_string_labels = False)


Run 1/5 started...
Epoch 1/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 235s 2s/step - accuracy: 0.5695 - loss: 1.0960
Epoch 2/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 225s 2s/step - accuracy: 0.7069 - loss: 0.6840
Epoch 3/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 226s 2s/step - accuracy: 0.7184 - loss: 0.6501
Epoch 4/1000
 20/101 ━━━━━━━━━━━━━━━━━━━━ 2:52 2s/step - accuracy: 0.7250 - loss: 0.6355

KeyboardInterrupt: 

In [7]:
run_model("shuttle_train.csv", "shuttle_test.csv", is_string_labels = False)

Run 1/5 started...
Epoch 1/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8407 - loss: 0.5563
Epoch 2/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9844 - loss: 0.0691
Epoch 3/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9875 - loss: 0.0508
Epoch 4/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9900 - loss: 0.0445
Epoch 5/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9905 - loss: 0.0398
Epoch 6/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9906 - loss: 0.0375
Epoch 7/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9936 - loss: 0.0243
Epoch 8/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9935 - loss: 0.0257
Epoch 9/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9942 - loss: 0.0204
Epoch 10/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9950 - loss: 0.0186
Epoch 11/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9947 - loss: 0.0193
Epoch 12/10

KeyboardInterrupt: 

In [ ]:
run_model("/kaggle/input/ma-datasets/covtype_train.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

Run 1/5 started...


KeyboardInterrupt: 

In [8]:
labels_map = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

run_model("/kaggle/input/ma-datasets/kdd_train.csv", "/kaggle/input/ma-datasets/kdd_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/5 started...
Run 2/5 started...
Run 3/5 started...
Run 4/5 started...
Run 5/5 started...

Aggregated Metrics:
test_loss: 0.2214
test_accuracy: 0.9580
balanced_accuracy: 0.0633
roc_auc: 0.5208
pr_auc: 0.9188
f2: 0.9501
mcc: 0.1666
0_precision: 0.9593
0_recall: 0.9992
0_f1-score: 0.9788
0_support: 194557.0000
1_precision: 0.0000
1_recall: 0.0000
1_f1-score: 0.0000
1_support: 3178.0000
2_precision: 0.1733
2_recall: 0.1799
2_f1-score: 0.1765
2_support: 2496.0000
3_precision: 0.1162
3_recall: 0.1499
3_f1-score: 0.1309
3_support: 2083.0000
4_precision: 0.0000
4_recall: 0.0000
4_f1-score: 0.0000
4_support: 463.0000
5_precision: 0.0000
5_recall: 0.0000
5_f1-score: 0.0000
5_support: 441.0000
6_precision: 0.0000
6_recall: 0.0000
6_f1-score: 0.0000
6_support: 204.0000
7_precision: 0.0000
7_recall: 0.0000
7_f1-score: 0.0000
7_support: 196.0000
8_precision: 0.0000
8_recall: 0.0000
8_f1-score: 0.0000
8_support: 53.0000
9_precision: 0.0000
9_recall: 0.0000
9_f1-score: 0.0000
9_support: 11.0000


In [9]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("/kaggle/input/ma-datasets/darknet_train.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/5 started...
Run 2/5 started...
Run 3/5 started...
Run 4/5 started...
Run 5/5 started...

Aggregated Metrics:
test_loss: 0.4617
test_accuracy: 0.8807
balanced_accuracy: 0.2308
roc_auc: 0.7651
pr_auc: 0.8296
f2: 0.8621
mcc: 0.3618
0_precision: 0.8886
0_recall: 0.9943
0_f1-score: 0.9378
0_support: 26862.0000
1_precision: 0.5486
1_recall: 0.3223
1_f1-score: 0.3862
1_support: 2657.0000
2_precision: 0.2568
2_recall: 0.3416
2_f1-score: 0.2884
2_support: 908.0000
3_precision: 0.3306
3_recall: 0.0594
3_f1-score: 0.0970
3_support: 522.0000
4_precision: 0.1139
4_recall: 0.0307
4_f1-score: 0.0484
4_support: 293.0000
5_precision: 0.3726
5_recall: 0.0260
5_f1-score: 0.0441
5_support: 269.0000
6_precision: 0.0919
6_recall: 0.0293
6_f1-score: 0.0444
6_support: 116.0000
7_precision: 0.1960
7_recall: 0.1057
7_f1-score: 0.1311
7_support: 53.0000
8_precision: 0.2202
8_recall: 0.1682
8_f1-score: 0.1851
8_support: 44.0000
accuracy: 0.8807
macro avg_precision: 0.3355
macro avg_recall: 0.2308
macro avg